# Defining a final DataSet

## Goal:
Recreate the final_dataset.ipynb

## Context:
Recreate the final_dataset, but each student will be considered as n-students where n is the number of semesters coursed. As an example, consider the student with ID 42. This student has been in the university for six semesters.
We will consider student 42 as six students: Student 42 from 1st semester, student 42 from 2nd semester, student 42 from 3rd semester, ... and student 42 from 6th semester.

This will allow the model to capture the behaviour of both dropout and non-dropout students in the early stages of their academic lives.
 

### Academic background
- **colegio**
- **puntaje icfes**
- **edad ingreso**
- **icfes puntaje especifico**
- **especialidad colegio icfes**
- **carrera matriculada**
- **Facultad**
- **genero**

### Academic performance
- **semestres cursados a la fecha** DONE
- **promedio academico a la fecha**
- **promedio ultimo semestre** DONE
- **Materias perdidas** DONE
- ~~**Materias cursadas**~~
- **Creditos cursados** DONE

All the features regarding academic performance will be affected by this approach!

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from datetime import datetime
import math
pd.set_option("display.max_columns", None)

We will start from two known files:
1. complete_student_performance
2. gpas_raw

These two files were originated from the final_dataset notebook. The first one has info for every subject that each student has coursed. The second one contains the GPAs for all students for each semester in a tidy format.

In [2]:
# Select the files
complete_student_performance_file = '../../data/processed/complete_student_performance.csv'
gpas_raw = '../../data/processed/gpas_raw.csv'

gpas_raw_df = pd.read_csv(gpas_raw, index_col=0).reset_index()
student_performance = pd.read_csv(complete_student_performance_file, index_col=0)

The following function is already known from other notebooks and helps convert columns data types.

We will use this function to adjust the columns data types for the student_performance DataFrame:

In [3]:
def change_type(df, cols, as_type):
    ''' 
    Takes a DataFrame and a list of columns and transform those columns to the selected format.
              
        Parameters
        ----------
        df: pandas.core.frame.DataFrame
            A pandas DataFrame where the columns will be converted into the desired format.
            
        cols: list
            A list with the name of the columns that will be converted into the desired format.
            
        as_type: {'str', 'float', 'int'}
            A list with the name of the columns that will be converted into the desired format.
            
        Returns
        -------
        dataframe: pandas.core.frame.DataFrame
            A DataFrame with the selected columsn converted into the desired format.
    ''' 
    if (as_type == 'str') or (as_type == 'int'):
        for col in cols:
            try:
                df[col] = df[col].astype(as_type)
            except: pass
        return df
    else:
        for col in cols:
            try:
                df[col] = df[col].str.replace(',','.')
                df[col] = df[col].astype('float64')
            except: pass
        return df

In [4]:
cat_cols = ['PERIODO_COHORTE', 'PERIODO_ACADEMICO', 'CODIGO', 'COD_MAT']
num_cols = ['NOTA_DEF']

student_performance = change_type(student_performance, cat_cols, 'str')
student_performance = change_type(student_performance, num_cols, 'float')


Let's create a new column boolean for the student_performance DataFrame that indicates whether a subject was or was not failed. Then, we can group that DataFrame by student's ID and period, adding the values of the new column:

In [39]:
student_performance['failed_flag'] = [True if i < 3 else False for i in student_performance['NOTA_DEF']]
failed_subjects_df = student_performance.groupby(['CODIGO','PERIODO_ACADEMICO'])[['failed_flag']].sum().reset_index()

Let's now create a copy of the same dataset but now the failed_flag column will be cumulative for each student:

In [40]:
def get_cumulative_failed(df):
    df['cumulative_failed'] = 0
    
    init_cumulative = 0
    codigo = 'sw'
    for index, row in df.iterrows():
        if codigo != df.loc[index, 'CODIGO']:
            codigo = df.loc[index, 'CODIGO']
            init_cumulative = 0
            
            df.loc[index, 'cumulative_failed'] = init_cumulative + df.loc[index, 'failed_flag']
            init_cumulative = df.loc[index, 'cumulative_failed']
        else:
            df.loc[index, 'cumulative_failed'] = init_cumulative + row['failed_flag'] 
            init_cumulative = df.loc[index, 'cumulative_failed']
            
    return df

failed_subjects_df = get_cumulative_failed(failed_subjects_df)
failed_subjects_df.head()

,CODIGO,PERIODO_ACADEMICO,failed_flag,cumulative_failed
0,11312001,201210,0.0,0.0
1,11312001,201220,0.0,0.0
2,11312001,201310,0.0,0.0
3,11312001,201320,0.0,0.0
4,11312001,201410,1.0,1.0


Let's do a similar process with a dataset stored in **'../../data/processed/gpas_grouped_by_student.csv'**. It's like the previous one but this one contains data about number of credits and GPAs per semester. Here we will try to get the cumulative number of credits coursed and the cumulative GPA over the semesters:

In [44]:
grouped_df = pd.read_csv('../../data/processed/gpas_grouped_by_student.csv', index_col=0)

def get_cumulative_credits(df):
    df['cumulative_credits'] = 0
    df['cumulative_gpa'] = 0
    
    init_cumulative = 0
    init_gpa = 0
    codigo = 'sw'
    
    for index, row in df.iterrows():
        if codigo != df.loc[index, 'CODIGO']:
            codigo = df.loc[index, 'CODIGO']
            init_cumulative = 0
            init_gpa = 0
            
            df.loc[index, 'cumulative_credits'] = init_cumulative + df.loc[index, 'NUM_CREDITOS']
            df.loc[index, 'cumulative_gpa'] = init_gpa + df.loc[index, 'GPA']
            
            init_cumulative = df.loc[index, 'cumulative_credits']
            init_gpa = df.loc[index, 'cumulative_gpa']
            
        else:
            df.loc[index, 'cumulative_credits'] = init_cumulative + row['NUM_CREDITOS']
            df.loc[index, 'cumulative_gpa'] = init_gpa + row['GPA']
            
            
            init_cumulative = df.loc[index, 'cumulative_credits']
            init_gpa = df.loc[index, 'cumulative_gpa']
            
    df['real_cumulative_gpa'] = df['cumulative_gpa']/df['cumulative_credits']
            
    return df

num_credits_df = get_cumulative_credits(grouped_df)

In [47]:
num_credits_df.head(5)

,CODIGO,PERIODO_ACADEMICO,NUM_CREDITOS,GPA,real_GPA,cumulative_credits,cumulative_gpa,real_cumulative_gpa
0,11312001,201210,15.0,53.7,3.58,15.0,53.7,3.580000
1,11312001,201220,12.0,47.7,3.98,27.0,101.4,3.755556
2,11312001,201310,10.0,34.8,3.48,37.0,136.2,3.681081
3,11312001,201320,16.0,56.4,3.53,53.0,192.6,3.633962
4,11312001,201410,14.0,46.1,3.29,67.0,238.7,3.562687


These dataframes will serve us for a later merge on our tidy dataset. So let's leave them here for now.
We can now focus on a new dataset called extended_gpas_raw.

This extended_gpas_raw dataset was generated by the **expand_gpas** notebook and represented each student as n different students where n is the number of semesters coursed as stated in the context.

**IMPORTANT**
1. The details of how this dataset was constructed are in the same notebook.
2. Most of the commented markdowns are the same as the final_dataset notebook since the processes are the same.

In [49]:
#extended_gpas_raw = pd.read_csv('extended_gpas_raw.xlsx')

It is not ideal to have one column for each period, but we can still get useful info from this table like the number of semesters and global GPA (we could have gotten this from the table before pivoting but let's do it from here).

The steps for achieving this are the following:

- Get the number of semesters coursed as the count of semesters where the GPA > 0.0
- Calculate the credits coursed with a table called "num_creds"
- Calculate the raw GPA with a table called "gpas_raw"
- Get the real GPA dividing the raw GPA by the gpas_raw
- Get the number of failed subjects

In [ ]:
# Getting the number of semesters:
gpas = extended_gpas_raw.copy()
gpas['n_semesters']= 0
for index, row in gpas.iterrows():
    for column in gpas.columns[0:-1]:
        if row[column] > 0:
            gpas.loc[index,'n_semesters'] = gpas.loc[index,'n_semesters'] + 1

In [ ]:
# Getting the number of credits per semesters as a new table called num_creds
num_creds = pd.pivot_table(grouped_df,values='NUM_CREDITOS',index=['CODIGO'],columns=['PERIODO_ACADEMICO'],aggfunc=np.sum)
num_creds['sum_credits']= 0
for index, row in num_creds.iterrows():
    for column in num_creds.columns[0:-1]:
        if row[column] > 0:
            num_creds.loc[index,'sum_credits'] = num_creds.loc[index,'sum_credits'] + row[column]

# Getting the raw GPA per semester as a new table called gpas_raw
gpas_raw = pd.pivot_table(grouped_df, values='GPA', index=['CODIGO'], columns=['PERIODO_ACADEMICO'], aggfunc=np.sum)
gpas_raw['sum_gpa']= 0
for index, row in gpas_raw.iterrows():
    for column in gpas_raw.columns[0:-1]:
        if row[column] > 0:
            gpas_raw.loc[index,'sum_gpa'] = gpas_raw.loc[index,'sum_gpa'] + row[column]
            
gpas = pd.merge(gpas, num_creds[['sum_credits']], how="left", left_on = "CODIGO", right_on = "CODIGO") 
gpas['GPA'] = np.array(gpas_raw['sum_gpa'])/np.array(num_creds['sum_credits'])